In [ ]:
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
palette = sns.color_palette()
color_matching = {
    "ELEET": palette[0],  # blue
    "Text-To-Table": palette[2],  # green
    "LLaMA-2 7B (ic)": palette[1],  # orange
    "LLaMA-2 7B (ft)": palette[3],  # red
    "gpt-3.5-turbo-0125": palette[4],  #purple
    "gpt-4-0613": palette[6]
}

methods = ["ELEET", "Text-To-Table", "LLaMA-2 7B (ic)", "LLaMA-2 7B (ft)"]
color_palette = [color_matching[m] for m in methods]

In [ ]:
from eleet.evaluate import RunDescription

queries = {'G_name(π_name,Assists,Blocks,Points,To…(Player))': 'Aggregation',
           'player_info ⨝ player_to_reports ⨝ Player': 'Join',
           'player_stats ∪ Player': 'Union',
           'π_name,Assists,Blocks,Points,To…(Player)': 'Scan',
           'π_name,Assists,Blocks,Points,To…(σ_Points=28[0.096](Player))': 'Selection',
           'G_name(π_name,Number of team assists,Losses,Total points,Wi…(Team))': 'Aggregation',
           'team_info ⨝ team_to_reports ⨝ Team': 'Join',
           'team_stats ∪ Team': 'Union',
           'π_name,Number of team assists,Losses,Total points,Wi…(Team)': 'Scan',
           'π_name,Number of team assists,Losses,Total points,Wi…(σ_Total points=102[0.063](Team))': 'Selection'}

RUNTIME_FILE = "../predictions/test/runtimes_eleet-t2t-llama-gpt-3.5-turbo-0125-gpt-4-0613-llama-ft-gpt-ft-rotowire-trex-aviation-corona-64-test-limit-4294967296.csv"

runtime = pd.read_csv(RUNTIME_FILE)
runtime["runtime_no_index"] = runtime["runtime"] - runtime["index_build_time"]
runtime["subset"] = runtime["query"].apply(lambda x: "Player" if "player" in x.lower() else "Team")
runtime = runtime[runtime["query"].apply(lambda x: x in queries)]
runtime["method"] = runtime["method"].apply(lambda x: "ft:gpt-3.5-turbo" if x.startswith("ft:") else x)

runtime.replace({"ELEET": "ELEET", "LLaMA": "LLaMA-2 7B (ic)", 4096: 3398, 16384: 3398, "LLaMA-FT": "LLaMA-2 7B (ft)", "gpt-3.5-turbo-0125": "gpt-3.5-turbo-0125 (ic)",
                 "gpt-4-0613": "gpt-4-0613 (ic)", **queries}, inplace=True)

# matplotlib.rcParams.update({'font.size': })
plt.rc('text', usetex=False)
sns.set_theme(style="whitegrid", font_scale=1.)


fig = plt.figure(figsize=(6, 1.5))
gs = fig.add_gridspec(nrows=1, ncols=2, hspace=0.1, wspace=0.1)
                         # gridspec_kw={"width_ratios": [1, 10, 1, 1, 10, 1, 1, 10], "wspace": 0.1})
axes = gs.subplots(sharey=True)

labels = list()
handles = list()

for i, subset in enumerate(("Player", "Team")):
    ax = axes[i]

    sns.barplot(runtime[runtime["subset"] == subset], y="runtime_no_index", x="query", hue="method", ax=ax,
                hue_order=methods, palette=sns.color_palette(color_palette),
                order=["Scan", "Join", "Union", "Selection", "Aggregation"])

    handles_ax, labels_ax = ax.get_legend_handles_labels()
    handles.extend(handles_ax)
    labels.extend(labels_ax)
    ax.get_legend().remove()
    ax.set_title(subset)

    ax.set_yscale("log")
    ax.set_yticks([1, 10, 100, 1000, 10000])
    ax.set_ylabel("Runtime [s]" if i == 0 else "")
    ax.set_xlabel(None)
    ax.set_xticklabels(ax.get_xticklabels(), rotation = 20)

hl_dict = {l.split("_")[0]: h for l, h in zip(labels, handles)}
print(hl_dict)
leg = fig.legend([hl_dict[l] for l in methods], methods, bbox_to_anchor=(0.9, 0.0), ncol=1, loc='lower left', borderaxespad=0., frameon=False)

plt.savefig("/home/murban/exp3.pdf", bbox_inches="tight")
plt.show()

# evt barplot: gesamtlaufzeit, mean f1
